# Import

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,regexp_replace
import pyarrow as pa
import pyarrow.parquet as pq

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.38.1 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::218606466161:role/LabRole
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 4a6e11d4-bd7d-48ab-87c5-0c5e47b7b034
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.38.1
--enable-glue-datacatalog true
Waiting for session 4a6e11d4-bd7d-48ab-87c5-0c5e47b7b034 to get into ready status...
Session 4a6e11d4-bd7d-48ab-87c5-0c5e47b7b034 has been created.



In [2]:
# Inicialize a sessão Spark
spark = SparkSession.builder \
    .appName("ReadS3DataReclamacoes") \
    .getOrCreate()

# Construa o caminho completo para o arquivo no S3
s3_path = f"s3://218606466161-raw/Reclamações/*.csv"
output_s3_path = "s3://218606466161-trusted/Reclamacoes/"

# Leia o arquivo CSV do S3
df_reclamacoes = spark.read.option("encoding", "UTF-8").csv(s3_path, header=True, inferSchema=True, sep=';')


In [3]:
#Tratando os nomes
df_reclamacoes = df_reclamacoes.withColumnRenamed("Quantidade total de clientes – CCS e SCR", "Quantidade total de clientes")

#Fazendo as agregações
df_reclamacoes = df_reclamacoes.groupby("Instituição financeira").agg({
        'Índice': 'min',
        'Quantidade total de reclamações': 'sum',
        'Quantidade total de clientes': 'mean'
    })

In [4]:
#Tratando colunas
df_reclamacoes_filtrado = df_reclamacoes.withColumn("Instituição financeira", regexp_replace(col('Instituição financeira'), " \(conglomerado\)", "")) \
                          .withColumnRenamed("Instituição financeira", "Nome") \
                          .withColumnRenamed("min(Índice)", "indice") \
                          .withColumnRenamed("sum(Quantidade total de reclamações)", "qtd_total_reclamacoes") \
                          .withColumnRenamed("avg(Quantidade total de clientes)", "qtd_total_clientes")

In [5]:
df_reclamacoes_filtrado.show()

+--------------------+------+---------------------+--------------------+
|                Nome|indice|qtd_total_reclamacoes|  qtd_total_clientes|
+--------------------+------+---------------------+--------------------+
|                 BBC|      |                   20|              5929.0|
|FORTBRASIL ADMINI...|      |                   28|            637834.0|
|            ORIGINAL| 14,35|                10168|2.5494336285714287E7|
|BANCO DA CHINA BR...|      |                    3|   492.6666666666667|
|BANCO DE LAGE LAN...|      |                    4|            35777.25|
|BBVA BRASIL BANCO...|      |                    1|                 0.0|
|         BNP PARIBAS|125,22|                 4624|  2211990.5714285714|
|   BNQI - FINANCEIRO|      |                  320|                 0.0|
|FISERV DO BRASIL ...|      |                    2|                93.0|
|          GRUPO ZEMA|      |                   88|  474978.14285714284|
|SF3 CRÉDITO, FINA...|      |                    9|

In [6]:
df_reclamacoes_filtrado.write.format("parquet").mode("overwrite").option("path", f"{output_s3_path}").save()